In [4]:
import json
from pathlib import Path
from soundevent import data, io
from collections import defaultdict
from tensorflow._api.v2.lite import Interpreter
from audioclass import SimpleIterator
from audioclass.models.birdnet import BirdNET
import pandas as pd
from tqdm import tqdm

In [ ]:
# Load in the annotation project

In [6]:
fp2_annotation_path = Path("SM4_6s_2024-07-18T00_26_05.155132.json")
with open(fp2_annotation_path) as file:
    annotation_contents = json.load(file)

print(json.dumps(annotation_contents, indent=2))

{
  "version": "1.1.0",
  "created_on": "2024-07-18T00:26:05.155132",
  "data": {
    "uuid": "b2f14a51-fa5d-4224-b1cc-07868ae33277",
    "collection_type": "annotation_project",
    "users": [
      {
        "uuid": "63e6952b-1e95-440b-9fbb-6e0aa03b4ecb",
        "username": "hhhyt",
        "email": "yuting.huang.23@ucl.ac.uk",
        "name": "Yuting Huang",
        "institution": null
      }
    ],
    "tags": [
      {
        "id": 0,
        "key": "n/a",
        "value": "nothing"
      },
      {
        "id": 1,
        "key": "environment",
        "value": "rain or wind"
      },
      {
        "id": 2,
        "key": "sound",
        "value": "bird"
      },
      {
        "id": 3,
        "key": "sound",
        "value": "unknown"
      },
      {
        "id": 4,
        "key": "sound",
        "value": "sunbird"
      },
      {
        "id": 5,
        "key": "sounds",
        "value": "parrot"
      },
      {
        "id": 6,
        "key": "sound",
        "valu

In [8]:
fp2_sample = io.load(fp2_annotation_path, type="annotation_project")
print(repr(fp2_sample))

AnnotationProject(created_on=datetime.datetime(2024, 5, 30, 15, 31, 35, 587708), name='SM4_6s', description='The clips recorded by sm04 device that requires annotations for the projcet.', instructions='2 clips of 6s extracted from each recording')


In [10]:
# view the clip annotations
for clip_annotation in fp2_sample.clip_annotations:
    clip = clip_annotation.clip
    recording = clip.recording
    print(
        f"* Recording {recording.path} [from "
        f"{clip.start_time:.3f}s to {clip.end_time:.3f}s]"
    )
    print(
        f"\t{len(clip_annotation.sound_events)} sound event annotations found"
    )
    for annotation in clip_annotation.sound_events:
        sound_event = annotation.sound_event
        print(f"\t+ Sound event ")
        for tag in annotation.tags:
            print(f"\t\t- {tag}")
    print("")

* Recording C:\Users\dorah\OneDrive\妗岄潰\seychelles_parrot_data\SM04 (FP2)\2020\SM04_06_2020\FP2_20200607_130000.wav [from 102.000s to 108.000s]
	0 sound event annotations found

* Recording C:\Users\dorah\OneDrive\妗岄潰\seychelles_parrot_data\SM04 (FP2)\2019\SM04_12_2019\SM04_20191217_170000.wav [from 102.000s to 108.000s]
	0 sound event annotations found

* Recording C:\Users\dorah\OneDrive\妗岄潰\seychelles_parrot_data\SM04 (FP2)\2019\SM04_10_2019\SM04_20191005_070000.wav [from 198.000s to 204.000s]
	0 sound event annotations found

* Recording C:\Users\dorah\OneDrive\妗岄潰\seychelles_parrot_data\SM04 (FP2)\2021\SM04_02_2021\FP2_20210212_060000.wav [from 42.000s to 48.000s]
	0 sound event annotations found

* Recording C:\Users\dorah\OneDrive\妗岄潰\seychelles_parrot_data\SM04 (FP2)\2020\SM04_07_2020\FP2_20200723_120000.wav [from 120.000s to 126.000s]
	0 sound event annotations found

* Recording C:\Users\dorah\OneDrive\妗岄潰\seychelles_parrot_data\SM04 (FP2)\2019\SM04_10_2019\SM04_20191011_1300

In [12]:
# save the annotation project
io.save(fp2_sample, "fp2_sample.json")

In [18]:
# 2) select the annotated clips

In [14]:
# create functions
def load_annotation_file(annotation_file):
    return io.load(annotation_file, type="annotation_project")


def has_issue(task):
    for badge in task.status_badges:
        if badge.state == data.AnnotationState.rejected:
            return True

    return False


def is_complete(task):
    for badge in task.status_badges:
        if badge.state == data.AnnotationState.completed:
            return True

    return False


def select_clips(annotation_project, complete=None, issue=None):
    tasks = [task for task in annotation_project.tasks]

    if complete is not None:
        tasks = [task for task in tasks if is_complete(task) == complete]

    if issue is not None:
        tasks = [task for task in tasks if has_issue(task) == issue]

    clips = {task.clip.uuid for task in tasks}
    return [
        clip_annotation
        for clip_annotation in annotation_project.clip_annotations
        if clip_annotation.clip.uuid in clips
    ]

path = "fp2_sample.json"
annotation_project = load_annotation_file(path)
complete_clips = select_clips(annotation_project, complete=True, issue=False)

In [16]:
# view the completely annotated clips
for clip_annotation in complete_clips:
    clip = clip_annotation.clip
    recording = clip.recording
    print(
        f"* Recording {recording.path} [from "
        f"{clip.start_time:.3f}s to {clip.end_time:.3f}s]"
    )

* Recording C:\Users\dorah\OneDrive\妗岄潰\seychelles_parrot_data\SM04 (FP2)\2020\SM04_06_2020\FP2_20200607_130000.wav [from 102.000s to 108.000s]
* Recording C:\Users\dorah\OneDrive\妗岄潰\seychelles_parrot_data\SM04 (FP2)\2019\SM04_12_2019\SM04_20191217_170000.wav [from 102.000s to 108.000s]
* Recording C:\Users\dorah\OneDrive\妗岄潰\seychelles_parrot_data\SM04 (FP2)\2019\SM04_10_2019\SM04_20191005_070000.wav [from 198.000s to 204.000s]
* Recording C:\Users\dorah\OneDrive\妗岄潰\seychelles_parrot_data\SM04 (FP2)\2021\SM04_02_2021\FP2_20210212_060000.wav [from 42.000s to 48.000s]
* Recording C:\Users\dorah\OneDrive\妗岄潰\seychelles_parrot_data\SM04 (FP2)\2020\SM04_07_2020\FP2_20200723_120000.wav [from 120.000s to 126.000s]
* Recording C:\Users\dorah\OneDrive\妗岄潰\seychelles_parrot_data\SM04 (FP2)\2019\SM04_10_2019\SM04_20191011_130000.wav [from 246.000s to 252.000s]
* Recording C:\Users\dorah\OneDrive\妗岄潰\seychelles_parrot_data\SM04 (FP2)\2019\SM04_05_2019\SM04_20190528_180000.wav [from 108.000s to 

In [18]:
len(complete_clips)

996

In [41]:
# 3) Extract features
 # Load latest annotations
annotation_project = io.load("fp2_sample.json")

In [43]:
# Load the birdnet model into the notebook!
model = BirdNET.load()

In [45]:
# as all 990 clips have been completely annotated, no need to remove any clips from the original json
selected_clip_annotations = annotation_project.clip_annotations[:996]

In [47]:
# check the audio dir and path
prefix = Path.home() / "OneDrive" / "桌面" / "seychelles_parrot_data"
print(f"Prefix exists: {prefix.exists()}")
audio_dir = Path("SM04 (FP2)")
print(f"audio dir: {audio_dir.exists()}")

Prefix exists: True
audio dir: False


In [49]:
# fix the path

for clip_annotation in selected_clip_annotations:
    clip = clip_annotation.clip
    recording = clip.recording

    # Only fix the path if not previously done
    recording.path = Path(str(recording.path).replace("妗岄潰","桌面"))

    if not recording.path.exists():
        raise ValueError(f"File not found {recording.path}")

In [51]:
# view the infos using an example clip
example_clip = selected_clip_annotations[0].clip

print("Clips contain info about when they start and end: ", clip.start_time, clip.end_time)

example_outputs = model.process_clip(example_clip)

print("The output contains two processed clips", len(example_outputs))

example_processed_clip = example_outputs[0]

print("Processed clips might be a subclip of the input: ", example_processed_clip.clip.start_time, example_processed_clip.clip.end_time) 

print("Processed clips can contain extracted features. In this case: ", len(example_processed_clip.features))

print("The first feature:  ", example_processed_clip.features[0])

Clips contain info about when they start and end:  186.0 192.0
The output contains two processed clips 2
Processed clips might be a subclip of the input:  102.0 105.0
Processed clips can contain extracted features. In this case:  1024
The first feature:   name='BirdNET_0' value=0.11091137677431107


In [53]:
# Process all clips in a loop
# Process all clips in a loop
def extract_tags_info(tags):
    return {tag.value: True for tag in tags}


def extract_task_info(annotation_task):
    ready = False
    issues = False

    for badge in annotation_task.status_badges:
        if badge.state == data.AnnotationState.completed:
            ready = True
            continue

        if badge.state == data.AnnotationState.rejected:
            issues = True
            continue

    return {"ready": ready, "issues": issues}


def extract_clip_annotation_info(clip_annotation):
    return {
        "annotation_id": str(clip_annotation.uuid),
        **extract_tags_info(clip_annotation.tags),
    }


def extract_feature_info(processed_clip):
    return {
        "path": processed_clip.clip.recording.path,
        "clip_id": processed_clip.clip.uuid,
        "start_time": processed_clip.clip.start_time,
        "end_time": processed_clip.clip.end_time,
        **{feature.name: feature.value for feature in processed_clip.features},
    }


# Process all clips in a loop
outputs = []
for clip_annotation in tqdm(selected_clip_annotations):
    clip = clip_annotation.clip

    # Its a good idea to wrap this processing bit in a try-except block
    # so that we can safely skip the clips that failed to process.
    try:
        processed_clips = model.process_clip(clip)

        # NOTE: If the clip is long the model will split it into multiple clips
        # and return a list of processed clips.
        for processed_clip in processed_clips:
            outputs.append(
                {
                    **extract_feature_info(processed_clip),
                    **extract_clip_annotation_info(clip_annotation),
                }
            )

    except Exception as e:
        print(f"Failed to process clip: {clip}. Error: {e}")
        continue

100%|████████████████████████████████████████████████████████████████████████████████| 996/996 [02:55<00:00,  5.68it/s]


In [55]:
# creat a pandas dataframe 
df = pd.DataFrame(outputs)

In [57]:
df

,path,clip_id,start_time,end_time,BirdNET_0,BirdNET_1,BirdNET_2,BirdNET_3,BirdNET_4,BirdNET_5,...,nothing,rain or wind,bird,unknown,sunbird,parrot,human,drops,frog,bulbul
0,C:\Users\dorah\OneDrive\桌面\seychelles_parrot_d...,cd175e4c-9bc2-466f-977b-a2ea9aa9bac7,102.0,105.0,0.110911,0.471305,0.000000,0.424498,0.258880,0.183433,...,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,C:\Users\dorah\OneDrive\桌面\seychelles_parrot_d...,6c935faf-7176-4e17-908c-e84204bee97c,105.0,108.0,0.063407,0.210319,0.092212,0.534250,0.584597,0.275086,...,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,C:\Users\dorah\OneDrive\桌面\seychelles_parrot_d...,8b31fd5b-6f61-4cf4-a284-31917fef65ff,102.0,105.0,0.000000,0.166991,0.022748,0.048599,0.416390,0.736442,...,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,C:\Users\dorah\OneDrive\桌面\seychelles_parrot_d...,d9dd3653-5739-4a40-be05-baefc4944d48,105.0,108.0,0.000000,0.213385,0.046879,0.110814,0.588934,0.519770,...,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,C:\Users\dorah\OneDrive\桌面\seychelles_parrot_d...,69aa5e29-15db-438f-a72e-b9335deea814,198.0,201.0,0.000000,0.113747,0.331943,0.000000,1.415365,0.302722,...,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1987,C:\Users\dorah\OneDrive\桌面\seychelles_parrot_d...,cfcbbc8d-e020-4a0f-8b44-8a0038d3e1fd,3.0,6.0,0.104903,0.000000,0.281769,0.480939,0.563529,0.248807,...,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1988,C:\Users\dorah\OneDrive\桌面\seychelles_parrot_d...,571253fe-18aa-49d9-aa9e-69fd466d8f91,162.0,165.0,0.106085,0.149327,0.001414,0.880642,0.303852,0.312123,...,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1989,C:\Users\dorah\OneDrive\桌面\seychelles_parrot_d...,29a4c0e8-451a-47d2-b87d-f6c9be812a1f,165.0,168.0,0.061831,0.379252,0.000000,0.686228,0.379335,0.308193,...,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990,C:\Users\dorah\OneDrive\桌面\seychelles_parrot_d...,dc866629-13d8-43f2-b84b-69430f92ad20,186.0,189.0,0.115977,0.220699,0.031106,0.317713,0.434829,0.055949,...,NaN,True,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN


In [59]:
df.to_csv("full_fp2_features.csv")

In [61]:
# compute the average features for two subclips
full_fp2_features = pd.read_csv('full_fp2_features.csv', low_memory=False)
full_fp2_features.drop(columns = ['clip_id', 'Unnamed: 0'])
full_fp2_features.columns

Index(['Unnamed: 0', 'path', 'clip_id', 'start_time', 'end_time', 'BirdNET_0',
       'BirdNET_1', 'BirdNET_2', 'BirdNET_3', 'BirdNET_4',
       ...
       'nothing', 'rain or wind', 'bird', 'unknown', 'sunbird', 'parrot',
       'human', 'drops', 'frog', 'bulbul'],
      dtype='object', length=1040)

In [63]:
# Define columns that are categorical (factors)
categorical_columns = ['path', 'nothing', 'rain or wind', 'bird', 'unknown', 'sunbird', 'parrot',
       'human', 'drops', 'frog', 'bulbul']  # Add any other categorical columns here

# Identify numeric columns for averaging
numeric_columns = full_fp2_features.select_dtypes(include=[float, int]).columns

# Remove categorical and time columns from numeric columns list if they are included
numeric_columns = [col for col in numeric_columns if col not in categorical_columns + ['start_time', 'end_time','annotation_id']]

# Group by 'annotation_id' and compute the average of each numeric feature column
fused_features_numeric = full_fp2_features.groupby('annotation_id')[numeric_columns].mean().reset_index()

# For categorical columns, we can keep the first instance (assuming consistency)
# Exclude 'annotation_id' here to avoid duplication
fused_features_categorical = full_fp2_features.groupby('annotation_id')[categorical_columns].first().reset_index()

# For start_time and end_time,custom logic
start_times = full_fp2_features.groupby('annotation_id')['start_time'].first().reset_index()
end_times = full_fp2_features.groupby('annotation_id')['end_time'].last().reset_index()

# Merge numeric, categorical, and time DataFrames
fused_features = fused_features_numeric.merge(fused_features_categorical, on='annotation_id')
fused_features = fused_features.merge(start_times, on='annotation_id')
fused_features = fused_features.merge(end_times, on='annotation_id')
fused_features.drop(columns = ['Unnamed: 0'])

,annotation_id,BirdNET_0,BirdNET_1,BirdNET_2,BirdNET_3,BirdNET_4,BirdNET_5,BirdNET_6,BirdNET_7,BirdNET_8,...,bird,unknown,sunbird,parrot,human,drops,frog,bulbul,start_time,end_time
0,003c0ab1-5aa1-4a48-ac33-119614c580cf,0.142146,0.022985,0.152949,0.251759,0.933654,0.388380,0.224773,0.334973,0.224981,...,None,True,None,None,None,None,None,None,24.0,30.0
1,0051ee9d-13ff-445a-868c-618ff2245cc2,0.053371,0.158845,0.005508,0.085640,1.023403,0.067680,0.620847,0.225151,0.238478,...,None,None,None,None,None,None,None,None,216.0,222.0
2,00aeab3e-bf1c-4f20-95c1-bcafcd326223,0.099882,0.143481,0.167155,0.290320,0.727669,0.523383,0.680426,0.198893,0.175511,...,True,None,None,None,None,None,None,None,162.0,168.0
3,01164aed-b0c9-4904-9ca7-14e93cd919eb,0.148729,0.152580,0.127707,0.317819,0.567609,0.201774,0.721012,0.135229,0.290836,...,None,None,None,None,None,None,None,None,276.0,282.0
4,01537369-ccf3-4380-b023-ebff4aea4672,0.125596,0.138627,0.402416,0.056818,1.073699,0.289138,0.207970,0.197457,0.220144,...,None,None,None,True,None,None,None,None,156.0,162.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
991,ff678209-b11d-4aa7-91c6-bc31afb77fb8,0.154141,0.170377,0.186990,0.744196,0.583663,0.202624,0.783130,0.007261,0.397042,...,None,True,None,None,None,None,None,None,282.0,288.0
992,ff7064e0-530a-4f13-b4e8-407c2b1e4705,0.115331,0.170805,0.137159,0.002068,0.702275,0.096649,1.043585,0.166812,0.152791,...,None,True,None,None,None,None,None,None,234.0,240.0
993,ff9a9131-2420-43a5-9dc9-054c301ff9df,0.071602,0.466810,0.149987,0.319607,0.325069,0.054760,0.973148,0.145510,0.434508,...,None,None,None,None,None,None,None,None,180.0,186.0
994,ff9fc276-185d-4fad-befc-78b5fb499f0a,0.180791,0.217682,0.247041,0.270837,0.566222,0.147767,0.702554,0.171368,0.257313,...,None,True,None,None,None,None,None,None,132.0,138.0


In [65]:
# save the fused features in to csv
fused_features.to_csv("full_fp2_features.csv")

In [67]:
# dropping unwanted columns and add site information for the ml development
ml_features = fused_features.drop(columns = ['Unnamed: 0','nothing', 'rain or wind', 'bird', 'unknown', 'sunbird','human', 'drops', 'frog', 'bulbul'])
ml_features['site']='fp2'
ml_features

,annotation_id,BirdNET_0,BirdNET_1,BirdNET_2,BirdNET_3,BirdNET_4,BirdNET_5,BirdNET_6,BirdNET_7,BirdNET_8,...,BirdNET_1019,BirdNET_1020,BirdNET_1021,BirdNET_1022,BirdNET_1023,path,parrot,start_time,end_time,site
0,003c0ab1-5aa1-4a48-ac33-119614c580cf,0.142146,0.022985,0.152949,0.251759,0.933654,0.388380,0.224773,0.334973,0.224981,...,0.275527,0.616276,0.167029,0.097635,0.975522,C:\Users\dorah\OneDrive\桌面\seychelles_parrot_d...,None,24.0,30.0,fp2
1,0051ee9d-13ff-445a-868c-618ff2245cc2,0.053371,0.158845,0.005508,0.085640,1.023403,0.067680,0.620847,0.225151,0.238478,...,0.355176,1.366054,0.107057,0.136292,1.428130,C:\Users\dorah\OneDrive\桌面\seychelles_parrot_d...,None,216.0,222.0,fp2
2,00aeab3e-bf1c-4f20-95c1-bcafcd326223,0.099882,0.143481,0.167155,0.290320,0.727669,0.523383,0.680426,0.198893,0.175511,...,0.569690,0.252652,0.030627,0.000000,0.376758,C:\Users\dorah\OneDrive\桌面\seychelles_parrot_d...,None,162.0,168.0,fp2
3,01164aed-b0c9-4904-9ca7-14e93cd919eb,0.148729,0.152580,0.127707,0.317819,0.567609,0.201774,0.721012,0.135229,0.290836,...,0.509290,0.342640,0.262395,0.000000,0.889886,C:\Users\dorah\OneDrive\桌面\seychelles_parrot_d...,None,276.0,282.0,fp2
4,01537369-ccf3-4380-b023-ebff4aea4672,0.125596,0.138627,0.402416,0.056818,1.073699,0.289138,0.207970,0.197457,0.220144,...,0.287313,0.558491,1.201600,0.194651,1.367779,C:\Users\dorah\OneDrive\桌面\seychelles_parrot_d...,True,156.0,162.0,fp2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
991,ff678209-b11d-4aa7-91c6-bc31afb77fb8,0.154141,0.170377,0.186990,0.744196,0.583663,0.202624,0.783130,0.007261,0.397042,...,0.499021,0.473640,0.335034,0.000413,1.471378,C:\Users\dorah\OneDrive\桌面\seychelles_parrot_d...,None,282.0,288.0,fp2
992,ff7064e0-530a-4f13-b4e8-407c2b1e4705,0.115331,0.170805,0.137159,0.002068,0.702275,0.096649,1.043585,0.166812,0.152791,...,0.345015,0.643588,0.207369,0.100199,1.365753,C:\Users\dorah\OneDrive\桌面\seychelles_parrot_d...,None,234.0,240.0,fp2
993,ff9a9131-2420-43a5-9dc9-054c301ff9df,0.071602,0.466810,0.149987,0.319607,0.325069,0.054760,0.973148,0.145510,0.434508,...,0.958800,0.499845,0.500082,0.030717,1.466951,C:\Users\dorah\OneDrive\桌面\seychelles_parrot_d...,None,180.0,186.0,fp2
994,ff9fc276-185d-4fad-befc-78b5fb499f0a,0.180791,0.217682,0.247041,0.270837,0.566222,0.147767,0.702554,0.171368,0.257313,...,0.286647,0.337338,0.145185,0.029964,1.026939,C:\Users\dorah\OneDrive\桌面\seychelles_parrot_d...,None,132.0,138.0,fp2


In [69]:
ml_features.to_csv("ml_fp2_features.csv")